In [ ]:
from caen_felib import device

## Connect

In [ ]:
dig = device.connect('dig2://caendgtz-eth-16218.caen.local/')


## Reset

dig.cmd.reset()

## Example of node browsing

In [ ]:

par_name = 'cupver'
print(f'Info of parameter {par_name}:')

for child in dig.par.get_node(f'/{par_name}').child_nodes:
    print(f'\t{child.name} == {child.value}')

## Configure digitizer

In [21]:
reclen = 102400

dig.set_value('/par/TestPulsePeriod', '1000')
dig.set_value('/par/TestPulseWidth', '16')
dig.set_value('/par/AcqTriggerSource', 'TestPulse|ITLA')
dig.set_value('/par/RecordLengthS', f'{reclen}')
dig.set_value('/par/PreTriggerS', '128')

dig.ch[0].par.SamplesOverThreshold.value = '16'
dig.ch[0].par.ITLConnect.value = 'ITLA'
dig.ch[0].par.TriggerThr.value = '8000'
dig.ch[0].par.TriggerThrMode.value = 'Absolute'
dig.ch[0].par.SelfTriggerEdge.value = 'Fall'

for ch in dig.ch.child_nodes:
	ch.set_value(f'/par/DCOffset', f'{0 + int(ch.name)}')
	ch.set_value(f'/par/WaveDataSource', 'adc_data')

dig.set_value('/endpoint/par/activeendpoint', 'scope')

## Test waveform plot

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.plot()
plt.show()